In [9]:
import re
import pandas as pd
from tqdm.auto import tqdm
from huggingface_hub import InferenceClient, login
from dotenv import load_dotenv
import os

In [2]:
tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)                  

In [3]:
load_dotenv("../.env")
token = os.getenv("HUGGINGFACE_TOKEN")
login(token)

In [4]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

In [5]:

# Test your LLM client
llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

'\n\nI’m good, thanks. I’m just about to go to the gym.'

In [17]:
ratings = pd.read_json("../data/feedback_train.json")

In [18]:
ratings.head()

,question,passage,feedback,rating,domain
0,How do I get help finding a job?,"{'passage_id': 140, 'source': 'Australia', 'uri': 'https://www.dese.gov.au/covid-19/job-seekers', 'reference_type': 'Passage_only', 'reference': {'page_title': 'Coronavirus (COVID-19) information for job seekers', 'section_headers': ['Exisiting job seekers'], 'section_content': 'If you are a current job seeker or participant, this fact sheet provides important information about mutual obligation requirements, appointments with your provider, and what to do if you are self-isolating: Information for job seekers and participants If you are participating in the ParentsNext program, this fact sheet provides important information about your activities and appointments. Information for ParentsNext participants ParentsNext participants Frequently Asked Questions If you are a New Business Assistance with NEIS participant, these Frequently Asked Questions (FAQ) provides information about accessing the Coronavirus Supplement and what support is available during this time: New Business Assistance with NEIS participants - Frequently Asked Questions If you are a New Business Assistance with NEIS provider, these Frequently Asked Questions (FAQ) provides information about supporting NEIS participants during the Coronavirus situation. New Business Assistance with NEIS providers – Frequently Asked Questions *[NEIS]: New Enterprise Incentive Scheme', 'selection_span': None, 'section_content_html': '<p>If you are a current job seeker or participant, this fact sheet provides important information about mutual obligation requirements, appointments with your provider, and what to do if you are self-isolating:</p> <ul> <li><a href=""https://docs.employment.gov.au/node/47535"">Information for job seekers and participants</a></li> </ul> <p>If you are participating in the ParentsNext program, this fact sheet provides important information about your activities and appointments.</p> <ul> <li> <p><a href=""https://docs.employment.gov.au/node/47551"">Information for ParentsNext participants</a></p> </li> <li> <p><a href=""https://docs.employment.gov.au/node/47550"">ParentsNext participants Frequently Asked Questions</a></p> </li> </ul> <p>If you are a New Business Assistance with NEIS participant, these Frequently Asked Questions (FAQ) provides information about accessing the Coronavirus Supplement and what support is available during this time:</p> <ul> <li><a href=""https://docs.employment.gov.au/node/47552"">New Business Assistance with NEIS participants - Frequently Asked Questions</a></li> </ul> <p>If you are a New Business Assistance with NEIS provider, these Frequently Asked Questions (FAQ) provides information about supporting NEIS participants during the Coronavirus situation.</p> <ul> <li><a href=""http://​docs.employment.gov.au/node/47592"">New Business Assistance with NEIS providers – Frequently Asked Questions</a></li> </ul> <p>*[NEIS]: New Enterprise Incentive Scheme</p>'}}","[Has a link to detailed information about government programs for job seekers., This answer provides a link for job searches, which is good, but also provides links to unrelated sites.]","[Excellent, Could be Improved]",Australia
1,How do I get help finding a job?,"{'passage_id': 139, 'source': 'Australia', 'uri': 'https://www.dese.gov.au/covid-19/job-seekers', 'reference_type': 'Passage_only', 'reference': {'page_title': 'Coronavirus (COVID-19) information for job seekers', 'section_headers': ['Jobs Hub'], 'section_content': 'In this rapidly changing jobs market the Australian Government is supporting businesses and those Australians looking for work. While many businesses have been adversely affected by COVID-19 and are reducing their workforces, there are some areas of the economy which have an increased demand for workers. This includes jobs in health and care sectors, transport and logistics, some areas of retail, mining and mining services, manufacturing, agriculture and government sector

In [20]:
ratings["rating1"] = ratings["rating"].map(lambda x: x[0])
ratings["rating2"] = ratings["rating"].map(lambda x: x[1])

In [21]:
# filter for agreement
ratings = ratings[ratings["rating1"] == ratings["rating2"]]

In [22]:
len(ratings)

2701

In [23]:
ratings["feedback1"] = ratings["feedback"].map(lambda x: x[0])
ratings["feedback2"] = ratings["feedback"].map(lambda x: x[1])

/var/folders/7q/r4n3w1j16h3fqh7n1q00mmp80000gn/T/ipykernel_70863/3448224524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["feedback1"] = ratings["feedback"].map(lambda x: x[0])
/var/folders/7q/r4n3w1j16h3fqh7n1q00mmp80000gn/T/ipykernel_70863/3448224524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["feedback2"] = ratings["feedback"].map(lambda x: x[1])


In [24]:
ratings.head()

,question,passage,feedback,rating,domain,rating1,rating2,feedback1,feedback2
1,How do I get help finding a job?,"{'passage_id': 139, 'source': 'Australia', 'uri': 'https://www.dese.gov.au/covid-19/job-seekers', 'reference_type': 'Passage_only', 'reference': {'page_title': 'Coronavirus (COVID-19) information for job seekers', 'section_headers': ['Jobs Hub'], 'section_content': 'In this rapidly changing jobs market the Australian Government is supporting businesses and those Australians looking for work. While many businesses have been adversely affected by COVID-19 and are reducing their workforces, there are some areas of the economy which have an increased demand for workers. This includes jobs in health and care sectors, transport and logistics, some areas of retail, mining and mining services, manufacturing, agriculture and government sectors, among others. The Jobs Hub helps you find advertised vacancies.', 'selection_span': None, 'section_content_html': '<p>In this rapidly changing jobs market the Australian Government is supporting businesses and those Australians looking for work.</p> <p>While many businesses have been adversely affected by COVID-19 and are reducing their workforces, there are some areas of the economy which have an increased demand for workers.</p> <p>This includes jobs in health and care sectors, transport and logistics, some areas of retail, mining and mining services, manufacturing, agriculture and government sectors, among others.</p> <p>The <a href=""/node/94"">Jobs Hub helps you find advertised vacancies</a>.</p>'}}","[A link to a job search website is included, as well as a list of job sectors that are hiring at this time., Includes a link to a Jobs Hub page, which is be helpful for job seekers.]","[Excellent, Excellent]",Australia,Excellent,Excellent,"A link to a job search website is included, as well as a list of job sectors that are hiring at this time.","Includes a link to a Jobs Hub page, which is be helpful for job seekers."
5,"If I am in Australia on a worker holiday marker visa, can I apply for a second visa because I work in a critical COVID-19 area?","{'passage_id': 546, 'source': 'Australia', 'uri': 'https://covid19.homeaffairs.gov.au/frequently-asked-questions', 'reference_type': 'FAQ', 'reference': {'page_title': 'Frequently Asked Questions', 'section_headers': ['Working holiday makers', 'Can I extend my current visa to stay in Australia if I am unable to return to my home country?'], 'section_content': 'You can apply for a further WHM visa if you have undertaken specified work for three months on your first WHM visa or six months on your second WHM visa. The following industries and areas are approved as specified work for subclass 417 (Working Holiday) visa holders: plant and animal cultivation in regional Australia fishing and pearling in regional Australia tree farming and felling in regional Australia mining in regional Australia construction in regional Australia and bushfire recovery work in declared bushfire affected areas only, carried out after 31 July 2019. The following industries and areas are approved as specified work for subclass 462 (Work and Holiday) visa holders: plant and animal cultivation in northern Australia and other specified areas of regional Australia fishing and pearling in northern Australia only tree farming and felling in northern Australia only tourism and hospitality in northern Australia only construction in northern Australia and other specified areas of regional Australia and bushfire recovery work in declared bushfire affected areas only, carried out after 31 July 2019. You must apply for a new visa before your current visa expires. You may be eligible to be granted a bridging visa that will keep you lawful until a decision is made on your visa application. If you are granted a Temporary Activity (subclass 408) Australian Government Endorsed Event (AGEE) stream visa, work undertaken on this visa does not count as specified work for the purpose 

In [28]:
ratings_map = {"Excellent": 4, "Acceptable": 3, "Could be Improved": 2, "Bad": 1}

In [29]:
ratings["rating_num1"] = ratings["rating1"].map(lambda x: ratings_map[x])
ratings["rating_num2"] = ratings["rating2"].map(lambda x: ratings_map[x])


/var/folders/7q/r4n3w1j16h3fqh7n1q00mmp80000gn/T/ipykernel_70863/1998754536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["rating_num1"] = ratings["rating1"].map(lambda x: ratings_map[x])
/var/folders/7q/r4n3w1j16h3fqh7n1q00mmp80000gn/T/ipykernel_70863/1998754536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["rating_num2"] = ratings["rating2"].map(lambda x: ratings_map[x])


In [30]:
print("Correlation between 2 human raters:")
print(f"{ratings['rating_num1'].corr(ratings['rating_num2'], method='pearson'):.3f}")

Correlation between 2 human raters:
1.000


In [34]:
ratings["human_score"] = ratings["rating_num1"]

/var/folders/7q/r4n3w1j16h3fqh7n1q00mmp80000gn/T/ipykernel_70863/4205015136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["human_score"] = ratings["rating_num1"]


In [36]:
ratings.to_csv("../data/ratings.csv", index=False)

In [31]:
examples = ratings.groupby("rating_num1").sample(7, random_state=1214)


In [32]:
examples

question  \
1976                                                                                         What can I do to help people that are grieving?   
4040              Should I continue to work if my work needs me, even though I am classified as vulnerable? Will I be penalised if I do not?   
5463                                                                                                   Will I be safe at my reopened school?   
2219                                                                                                        What does community spread mean?   
1104                                                                                             Is the Great Barrier Reef Marine Park open?   
5636                                                            I've been vaccinated against BCG, am I also protected from getting COVID-19?   
5390                                              Who will get priority for our limited medical resources and equipment during the pandemic?   
78                                                                       What protocols do workplaces need to follow to keep everyone safer?   
208   Who decides if our early childhood education and care service has to close due to COVID-19, and what should we do after shutting down?   
472                                                                                         Can the covid19 event visa be granted to anyone?   
4367                                           I am a young carer and am under strain right now. What resources and help is available to me?   
239                                                       Are there any personal hygiene practices that can help stop the spread of COVID-19   
1975                                                                                         What can I do to help people that are grieving?   
670                                                                                      What programs can assist busy childcare facilities?   
712                                                                                              How soon can I apply for financial support?   
1877                                                      What should I do to implement quarantine in my correctional or detention facility?   
1225                                                     What restrictions have been placed on any cruise ship that has left a foreign port?   
662                                                                                      How is the government helping to keep credit going?   
2026                                                               How should I know whether I need to isolate myself or go into quarantine?   
4780                                                                               Are universities going to be making unconditional offers?   
2690                                                        When shipping specimens, should a responsible person be listed on the paperwork?   
3147                                                                   Should vulnerable children be expected to be in educational settings?   
5375                                                                                             What symptoms are associated with Covid-19?   
651                                                                                     What product should I use to disinfect hard surfaces   
5150                                                           Can suspected and confirmed COVID-19 cases be kept in the same hospital room?   
5330                                                   What are the requirements for the attendance of athletes who are competing in events?   
1434                                                    What could I do to protect my employees as a supervisor at a rail transit operation?   
4194                                                          What do 3rd trimester or at risk healt

In [33]:
display(examples.groupby("human_score").first())

question  \
human_score                                                                          
1                                  What can I do to help people that are grieving?   
2              What protocols do workplaces need to follow to keep everyone safer?   
3                                      How soon can I apply for financial support?   
4            Should vulnerable children be expected to be in educational settings?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     